# import required libraries

In [1]:
from datetime import datetime
from hts import HTSRegressor
from hts.utilities.load_data import load_hierarchical_sine_data
from pathlib import Path
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')

In [2]:
predictable_variables=['total',"Area1"] +  ['Dealer1','Dealer2','Dealer3','Dealer4','Dealer5','Dealer6','Dealer7']
# predictable_variables = ["total","TN1_TN1-Vellore"] + ['TN1_TN1-Vellore_80124',
#   'TN1_TN1-Vellore_10853',
#   'TN1_TN1-Vellore_10689',
#   'TN1_TN1-Vellore_11720',
#   'TN1_TN1-Vellore_11992',
#   'TN1_TN1-Vellore_11706',
#   'TN1_TN1-Vellore_13054']
exogenus_variables = ['week_of_month', 'peak_flag', 'festival_flag', 'covid_flag']
m =12

# Defining the hierarchy of the data

In [92]:
# hier = {'total': ['TN1_TN1-Vellore'],
#  'TN1_TN1-Vellore': ['TN1_TN1-Vellore_80124',
#   'TN1_TN1-Vellore_10853',
#   'TN1_TN1-Vellore_10689',
#   'TN1_TN1-Vellore_11720',
#   'TN1_TN1-Vellore_11992',
#   'TN1_TN1-Vellore_11706',
#   'TN1_TN1-Vellore_13054']}

In [3]:
# hier = {'total': ['Area1'],
#   'Area1' : ['Dealer1','Dealer2',
#   'Dealer3','Dealer4','Dealer5','Dealer6','Dealer7']}
hier = {'total': ["Area1"],
"Area1" : ['Dealer1',
'Dealer2',
'Dealer3',
'Dealer4',
'Dealer5',
'Dealer6',
'Dealer7'] }

# data for training models and model selection

In [4]:
data_path  = str(Path.cwd().parent)+ "/data/train_test_data.csv"
df = pd.read_csv(data_path)
df['start_of_week'] = pd.to_datetime(df['start_of_week'], infer_datetime_format=True) 
df['start_of_week'] = df['start_of_week'].dt.tz_localize(None)
df = df.set_index("start_of_week")
df = df[predictable_variables+exogenus_variables]
df.head()

,total,Area1,Dealer1,Dealer2,Dealer3,Dealer4,Dealer5,Dealer6,Dealer7,week_of_month,peak_flag,festival_flag,covid_flag
start_of_week,,,,,,,,,,,,,
2017-01-09,3,3,0,3,0,0,0,0,0,1,0,0,0
2017-01-16,3,3,1,0,0,0,0,0,2,2,0,0,0
2017-01-23,2,2,2,0,0,0,0,0,0,3,0,0,0
2017-01-30,4,4,0,3,0,0,0,0,1,4,0,0,0
2017-02-06,0,0,0,0,0,0,0,0,0,1,0,0,0


In [5]:
df.columns

Index(['total', 'Area1', 'Dealer1', 'Dealer2', 'Dealer3', 'Dealer4', 'Dealer5',
       'Dealer6', 'Dealer7', 'week_of_month', 'peak_flag', 'festival_flag',
       'covid_flag'],
      dtype='object')

In [6]:
data_path = str(Path.cwd().parent)+ "/data/fcst_input_data.csv"
fcst_input_data = pd.read_csv(data_path)
fcst_input_data['start_of_week'] = pd.to_datetime(fcst_input_data['fcst_weeks'],infer_datetime_format=True)
fcst_input_data['start_of_week'] = fcst_input_data['start_of_week'].dt.tz_localize(None)
fcst_input_data.drop(['fcst_weeks'],axis = 1,inplace=True)
fcst_input_data.set_index("start_of_week",inplace=True)
fcst_input_data = fcst_input_data[exogenus_variables]
fcst_input_data.head()

,week_of_month,peak_flag,festival_flag,covid_flag
start_of_week,,,,
2021-09-06,1,1,0,0
2021-09-13,2,1,0,0
2021-09-20,3,1,0,0
2021-09-27,4,1,0,0
2021-10-04,1,1,1,0


In [7]:
cd /Users/karanamramachaitanya/Downloads/cltv_space/forecasting/auto_hts_package/

/Users/karanamramachaitanya/Downloads/cltv_space/forecasting/auto_hts_package


# Importing Auto HTS regressor class

In [8]:
from auto_hts import Auto_HTS_Regressor
auto_hts_instance = Auto_HTS_Regressor(hier=hier)

Importing plotly failed. Interactive plots will not work.


In [9]:
auto_hts_instance.fit(df,exogenus_variables = exogenus_variables,predictable_variables=predictable_variables,m=12)

starting ARIM Model
starting HWSE Model
starting Propeht Model
Initial log joint probability = -10.9467
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       306.193   0.000121393       89.4867      0.9019      0.9019      124   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     112       306.251   0.000368559       97.6142   4.147e-06       0.001      193  LS failed, Hessian reset 
     163       306.284   1.98347e-06       75.6238   2.805e-08       0.001      295  LS failed, Hessian reset 
     185       306.284   1.18783e-08       82.8958      0.2026      0.6343      324   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
Initial log joint probability = -10.9467
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       306.193   0.000121393       89.4867      0.9019      0.9019   

In [10]:
output = auto_hts_instance.predict(fcst_input_data)

starting forecast refitting 
fcst_start_date 2021-09-06 00:00:00
fcst_end_date 2022-02-28 00:00:00
starting the refit for arima 
with exo exogeneous-m-12
with exo exogeneous-m-12
starting the refit for HWSE
starting the refit for PROP 
Initial log joint probability = -8.32517
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      48       350.222     0.0018409       82.7965   1.973e-05       0.001       98  LS failed, Hessian reset 
      99       350.405     5.328e-06       83.3347      0.3086      0.3086      173   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     107       350.671    0.00234836       76.8131   2.606e-05       0.001      224  LS failed, Hessian reset 
     142       350.849   3.09691e-05       83.4864   4.721e-07       0.001      303  LS failed, Hessian reset 
     199       350.857   6.52779e-07       71.1288           1           1      378   
    Iter      log prob        ||

In [11]:
output

,total,Area1,Dealer1,Dealer2,Dealer3,Dealer4,Dealer5,Dealer6,Dealer7
2021-09-06,80.0,0.0,7.0,24.0,7.0,7.0,6.0,8.0,20.0
2021-09-13,83.0,0.0,7.0,24.0,7.0,8.0,9.0,7.0,19.0
2021-09-20,83.0,0.0,8.0,24.0,7.0,7.0,11.0,8.0,19.0
2021-09-27,82.0,0.0,7.0,23.0,7.0,9.0,12.0,7.0,17.0
2021-10-04,79.0,0.0,8.0,24.0,4.0,8.0,9.0,8.0,17.0
2021-10-11,78.0,0.0,8.0,25.0,5.0,8.0,6.0,8.0,17.0
2021-10-18,80.0,0.0,8.0,26.0,5.0,8.0,9.0,8.0,17.0
2021-10-25,85.0,0.0,7.0,25.0,9.0,9.0,11.0,7.0,17.0
2021-11-01,86.0,0.0,9.0,26.0,8.0,10.0,6.0,9.0,19.0
2021-11-08,84.0,0.0,9.0,25.0,8.0,9.0,6.0,9.0,19.0


# compare with single forecasting model

In [105]:
cd '/Users/karanamramachaitanya/Downloads/cltv_space/forecasting/auto_hts_package/test'

/Users/karanamramachaitanya/Downloads/cltv_space/forecasting/auto_hts_package/test


In [106]:
data_path  = str(Path.cwd().parent)+ "/data/new_train_test_data.csv"
df = pd.read_csv(data_path)
df['start_of_week'] = pd.to_datetime(df['start_of_week'], infer_datetime_format=True) 
df['start_of_week'] = df['start_of_week'].dt.tz_localize(None)
df = df.set_index("start_of_week")
df = df[predictable_variables+exogenus_variables]
df.head()

,total,TN1_TN1-Vellore,TN1_TN1-Vellore_80124,TN1_TN1-Vellore_10853,TN1_TN1-Vellore_10689,TN1_TN1-Vellore_11720,TN1_TN1-Vellore_11992,TN1_TN1-Vellore_11706,TN1_TN1-Vellore_13054,week_of_month,peak_flag,festival_flag,covid_flag
start_of_week,,,,,,,,,,,,,
2017-01-09,3,3,0,3,0,0,0,0,0,1,0,0,0
2017-01-16,3,3,1,0,0,0,0,0,2,2,0,0,0
2017-01-23,2,2,2,0,0,0,0,0,0,3,0,0,0
2017-01-30,4,4,0,3,0,0,0,0,1,4,0,0,0
2017-02-06,0,0,0,0,0,0,0,0,0,1,0,0,0


In [107]:
def train_test_data(df: pd.DataFrame):
    """ This function splits the data into train and test datasets in the ratio of 80:20
    
    Args : 
        df( Pandas DataFrame ) :  it's a Pandas Dataframe of the time series with all the columns of the hierarchial time series

    Returns :
        train_data(Pandas DataFrame) : Time series data of all the nodes of hierarchial time series with 80 percent of total data
        test_data(Pandas DataFrame) : Time series data of all the nodes of hierarchial time series with 20 percent of total data

    Erros :
        Value Error : The Input pandas DataFrame should have atleast 12 rows, if not it will raise an error
    """
    if df.shape[0] >= 12 :
        train_sample_size = np.int(np.floor(df.shape[0]*0.8))
        train_data = df.iloc[:train_sample_size,: ]
        test_data = df.iloc[train_sample_size : df.shape[0], :]
        return train_data, test_data

In [108]:
train_data , test_data = train_test_data(df)

In [109]:
train_data = train_data[predictable_variables]
train_data

,total,TN1_TN1-Vellore,TN1_TN1-Vellore_80124,TN1_TN1-Vellore_10853,TN1_TN1-Vellore_10689,TN1_TN1-Vellore_11720,TN1_TN1-Vellore_11992,TN1_TN1-Vellore_11706,TN1_TN1-Vellore_13054
start_of_week,,,,,,,,,
2017-01-09,3,3,0,3,0,0,0,0,0
2017-01-16,3,3,1,0,0,0,0,0,2
2017-01-23,2,2,2,0,0,0,0,0,0
2017-01-30,4,4,0,3,0,0,0,0,1
2017-02-06,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2020-08-24,73,73,16,14,6,5,25,0,7
2020-08-31,99,99,10,26,5,3,29,0,26
2020-09-07,72,72,16,17,4,3,15,0,17


In [110]:
test_data.shape

(49, 13)

In [111]:
fcst_input_data = test_data[exogenus_variables]
fcst_input_data.head()

,week_of_month,peak_flag,festival_flag,covid_flag
start_of_week,,,,
2020-09-28,4,1,0,0
2020-10-05,1,1,1,0
2020-10-12,2,1,1,0
2020-10-19,3,1,1,0
2020-10-26,4,1,1,0


In [112]:
reg = HTSRegressor(model='prophet', revision_method='OLS')
reg.fit(df=train_data,nodes = hier)
final_ouput = reg.predict(steps_ahead=49)

Fitting models:   0%|          | 0/9 [00:00<?, ?it/s]Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Fitting models:   0%|          | 0/9 [00:00<?, ?it/s]Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed.

In [113]:
hts_regressor_output = final_ouput[-49:]
hts_regressor_output.shape

(49, 9)

In [114]:
cd /Users/karanamramachaitanya/Downloads/cltv_space/forecasting/auto_hts_package/

/Users/karanamramachaitanya/Downloads/cltv_space/forecasting/auto_hts_package


In [115]:
from auto_hts import Auto_HTS_Regressor
auto_hts_instance = Auto_HTS_Regressor(hier=hier)

In [116]:
auto_hts_instance.fit(train_data,exogenus_variables=[],predictable_variables=predictable_variables,m=12)

starting ARIM Model
starting HWSE Model
starting Propeht Model
Initial log joint probability = -5.97642
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       206.084   3.62135e-07       60.6559      0.6026      0.6026      129   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     121       206.084   1.15506e-07       78.1063   1.536e-09       0.001      199  LS failed, Hessian reset 
     129       206.084   8.64028e-09        74.529      0.2834      0.2834      210   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance
prop successful
Initial log joint probability = -5.97642
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       206.084   3.62135e-07       60.6559      0.6026      0.6026      129   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  N

In [ ]:
Auto_HTS_Regressor_output = auto_hts_instance.predict(fcst_input_data)
Auto_HTS_Regressor_output.shape

In [118]:
auto_hts_instance.hier

{'total': ['TN1_TN1-Vellore'],
 'TN1_TN1-Vellore': ['TN1_TN1-Vellore_80124',
  'TN1_TN1-Vellore_10853',
  'TN1_TN1-Vellore_10689',
  'TN1_TN1-Vellore_11720',
  'TN1_TN1-Vellore_11992',
  'TN1_TN1-Vellore_11706',
  'TN1_TN1-Vellore_13054']}

In [119]:
hts_regressor_output.head()

,total,TN1_TN1-Vellore,TN1_TN1-Vellore_80124,TN1_TN1-Vellore_10853,TN1_TN1-Vellore_10689,TN1_TN1-Vellore_11720,TN1_TN1-Vellore_11992,TN1_TN1-Vellore_11706,TN1_TN1-Vellore_13054
2020-09-22,39.109639,39.109639,-6.489788,-10.014390,52.717704,-5.269265,0.338587,8.067337,-0.240548
2020-09-23,38.733701,38.733701,-6.632239,-9.897423,52.302549,-5.126031,0.178958,8.153873,-0.245987
2020-09-24,38.357523,38.357523,-6.770053,-9.785237,51.893829,-4.992993,0.036377,8.221059,-0.245459
2020-09-25,37.985511,37.985511,-6.901290,-9.679102,51.496520,-4.872019,-0.085673,8.267690,-0.240616
2020-09-26,37.622165,37.622165,-7.024145,-9.580186,51.115566,-4.764639,-0.184252,8.293004,-0.233183


In [121]:
Auto_HTS_Regressor_output.head()

,total,TN1_TN1-Vellore,TN1_TN1-Vellore_80124,TN1_TN1-Vellore_10853,TN1_TN1-Vellore_10689,TN1_TN1-Vellore_11720,TN1_TN1-Vellore_11992,TN1_TN1-Vellore_11706,TN1_TN1-Vellore_13054
2020-09-28,61.0,61.0,0.0,24.0,3.0,3.0,16.0,2.0,17.0
2020-10-05,59.0,59.0,1.0,19.0,3.0,6.0,16.0,1.0,15.0
2020-10-12,60.0,60.0,1.0,23.0,2.0,5.0,15.0,1.0,17.0
2020-10-19,62.0,62.0,2.0,21.0,4.0,6.0,15.0,1.0,15.0
2020-10-26,65.0,65.0,2.0,23.0,5.0,5.0,15.0,1.0,17.0


In [122]:
test_data.head()

,total,TN1_TN1-Vellore,TN1_TN1-Vellore_80124,TN1_TN1-Vellore_10853,TN1_TN1-Vellore_10689,TN1_TN1-Vellore_11720,TN1_TN1-Vellore_11992,TN1_TN1-Vellore_11706,TN1_TN1-Vellore_13054,week_of_month,peak_flag,festival_flag,covid_flag
start_of_week,,,,,,,,,,,,,
2020-09-28,98,98,16,25,12,10,30,0,5,4,1,0,0
2020-10-05,70,70,11,14,1,7,26,0,11,1,1,1,0
2020-10-12,68,68,12,21,6,7,16,0,6,2,1,1,0
2020-10-19,85,85,7,32,6,6,22,0,12,3,1,1,0
2020-10-26,111,111,21,26,8,4,35,0,17,4,1,1,0


In [123]:
# required functions 
funct = lambda x: np.floor(abs(x)) if ((abs(x) - np.floor(abs(x))) <= 0.5) else np.ceil(abs(x))
funct1 = lambda x: 1 if x == 0 else x

In [132]:
# len(hts_regressor_output["total"]- test_data["total"] )
# - test_data["total"]

91

In [144]:
for col in df.columns[:-4]:
    print(f"column name is {col}-->", np.mean(np.abs( np.array(hts_regressor_output[col]) - np.array(test_data[col]))) )

column name is total--> 38.20068255688102
column name is TN1_TN1-Vellore--> 38.20068255688102
column name is TN1_TN1-Vellore_80124--> 16.340454086261317
column name is TN1_TN1-Vellore_10853--> 27.400533858318386
column name is TN1_TN1-Vellore_10689--> 48.53520762515711
column name is TN1_TN1-Vellore_11720--> 11.373904303032653
column name is TN1_TN1-Vellore_11992--> 11.576357727566522
column name is TN1_TN1-Vellore_11706--> 5.252890044635566
column name is TN1_TN1-Vellore_13054--> 12.636431619759007


In [139]:
for col in df.columns[:-4]:
    print(f"column name is {col}-->", np.mean(np.abs( np.array(Auto_HTS_Regressor_output[col]) - np.array(test_data[col]))) )

column name is total--> 36.44897959183673
column name is TN1_TN1-Vellore--> 36.44897959183673
column name is TN1_TN1-Vellore_80124--> 7.979591836734694
column name is TN1_TN1-Vellore_10853--> 9.224489795918368
column name is TN1_TN1-Vellore_10689--> 4.224489795918367
column name is TN1_TN1-Vellore_11720--> 3.163265306122449
column name is TN1_TN1-Vellore_11992--> 13.448979591836734
column name is TN1_TN1-Vellore_11706--> 5.755102040816326
column name is TN1_TN1-Vellore_13054--> 8.204081632653061
